In [1]:
from osgeo import gdal
import numpy as np
import pandas as pd
import richdem as rd
import os
import glob

Constantes

In [2]:
PATH = './data-sentinel-2'

Graficar la data

In [3]:
class DefaultMetaObj:
    geotransform = [0,1,0,0,0,-1]

class DataGenerator:
    def __init__(self, directory):
        files = []
        files.extend(glob.glob(os.path.join(directory, "pre_landslide_test_*.tif")))
        self.files = sorted(files)
    
    def __getitem__(self, index: int):   
        filename = self.files[index]
        fileId = os.path.split(filename)[1].replace("pre_landslide_test_", "").replace(".tif", "")
        gdal_data = gdal.Open(filename)
        bands = { gdal_data.GetRasterBand(i + 1).GetDescription(): i for i in range(gdal_data.RasterCount) }
        
        for band in bands.keys():
            bands[band] = gdal_data.ReadAsArray().astype(float)[bands[band]]

        return fileId, bands

    def __center_crop__(self, img, new_width=None, new_height=None):        

        width = img.shape[1]
        height = img.shape[0]

        if new_width is None:
            new_width = min(width, height)

        if new_height is None:
            new_height = min(width, height)

        left = int(np.ceil((width - new_width) / 2))
        right = width - int(np.floor((width - new_width) / 2))

        top = int(np.ceil((height - new_height) / 2))
        bottom = height - int(np.floor((height - new_height) / 2))

        if len(img.shape) == 2:
            center_cropped_img = img[top:bottom, left:right]
        else:
            center_cropped_img = img[top:bottom, left:right, ...]

        return center_cropped_img

    def getData(self):
        output = pd.DataFrame()
        for i in range(len(self.files)):
            dictData = dict()
            fileId, data = self.__getitem__(i)
            
            # Elevation
            elevation = rd.rdarray(data['elevation'], no_data=np.nan, meta_obj=DefaultMetaObj())


            # Slope (percentage)
            slope = rd.TerrainAttribute(elevation, attrib='slope_percentage', zscale=0.1)
            slope =  self.__center_crop__(slope, 5, 5)
            dictData.update({
                'slope_1': slope[0][0], 'slope_6': slope[0][1],  'slope_11': slope[0][2], 'slope_16': slope[0][3], 'slope_21': slope[0][4], 
                'slope_2': slope[1][0], 'slope_7': slope[1][1],  'slope_12': slope[1][2], 'slope_17': slope[1][3], 'slope_22': slope[1][4], 
                'slope_3': slope[2][0], 'slope_8': slope[2][1],  'slope_13': slope[2][2], 'slope_18': slope[2][3], 'slope_23': slope[2][4], 
                'slope_4': slope[3][0], 'slope_9': slope[3][1],  'slope_14': slope[3][2], 'slope_19': slope[3][3], 'slope_24': slope[3][4], 
                'slope_5': slope[4][0], 'slope_10': slope[4][1], 'slope_15': slope[4][2], 'slope_20': slope[4][3], 'slope_25': slope[4][4]
            })
            
            # Aspect
            aspect = rd.TerrainAttribute(elevation, attrib='aspect', zscale=0.1)
            aspect =  self.__center_crop__(aspect, 5, 5)
            dictData.update({
                'aspect_1': aspect[0][0], 'aspect_6': aspect[0][1],  'aspect_11': aspect[0][2], 'aspect_16': aspect[0][3], 'aspect_21': aspect[0][4], 
                'aspect_2': aspect[1][0], 'aspect_7': aspect[1][1],  'aspect_12': aspect[1][2], 'aspect_17': aspect[1][3], 'aspect_22': aspect[1][4], 
                'aspect_3': aspect[2][0], 'aspect_8': aspect[2][1],  'aspect_13': aspect[2][2], 'aspect_18': aspect[2][3], 'aspect_23': aspect[2][4], 
                'aspect_4': aspect[3][0], 'aspect_9': aspect[3][1],  'aspect_14': aspect[3][2], 'aspect_19': aspect[3][3], 'aspect_24': aspect[3][4], 
                'aspect_5': aspect[4][0], 'aspect_10': aspect[4][1], 'aspect_15': aspect[4][2], 'aspect_20': aspect[4][3], 'aspect_25': aspect[4][4]
            })
            
            # Curvature
            curvature = rd.TerrainAttribute(elevation, attrib='curvature', zscale=0.1)
            curvature =  self.__center_crop__(curvature, 5, 5)
            dictData.update({
                'curvature_1': curvature[0][0], 'curvature_6': curvature[0][1],  'curvature_11': curvature[0][2], 'curvature_16': curvature[0][3], 'curvature_21': curvature[0][4], 
                'curvature_2': curvature[1][0], 'curvature_7': curvature[1][1],  'curvature_12': curvature[1][2], 'curvature_17': curvature[1][3], 'curvature_22': curvature[1][4], 
                'curvature_3': curvature[2][0], 'curvature_8': curvature[2][1],  'curvature_13': curvature[2][2], 'curvature_18': curvature[2][3], 'curvature_23': curvature[2][4], 
                'curvature_4': curvature[3][0], 'curvature_9': curvature[3][1],  'curvature_14': curvature[3][2], 'curvature_19': curvature[3][3], 'curvature_24': curvature[3][4], 
                'curvature_5': curvature[4][0], 'curvature_10': curvature[4][1], 'curvature_15': curvature[4][2], 'curvature_20': curvature[4][3], 'curvature_25': curvature[4][4]
            })

            # Save in dataframe
            output = pd.concat([output, pd.DataFrame(dictData, index=[fileId])])
        return output
            

In [4]:
dg = DataGenerator(PATH)
data = dg.getData()

In [ ]:
data.to_csv('./results/data.csv')  